In [43]:
import os
from dotenv import load_dotenv
load_dotenv() # Load environment variables from a .env file if present

groq_api_key = os.getenv("GROQ_API_KEY")

In [66]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)


In [45]:
from langchain_core.messages import HumanMessage
ans=model.invoke([HumanMessage(content="Hii, My name is Yash")])
ans

AIMessage(content="Hi Yash, it's nice to meet you! 😊\n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_time': 0.047272727, 'prompt_time': 0.00125728, 'queue_time': 0.25219033, 'total_time': 0.048530007}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b76f8234-3dad-47da-a760-84207a99b6d0-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41})

In [46]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hii, My name is Yash"),
    AIMessage(content="Hello Yash! It's nice to meet you. 👋\n\nWhat can I do for you today?\n"),
    HumanMessage(content="can you tell my name?")
])

AIMessage(content='You told me your name is Yash! 😊 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 53, 'total_tokens': 77, 'completion_time': 0.043636364, 'prompt_time': 0.00170799, 'queue_time': 0.25320953, 'total_time': 0.045344354}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--38720821-fa21-4d99-a112-ee9fa57ad1e5-0', usage_metadata={'input_tokens': 53, 'output_tokens': 24, 'total_tokens': 77})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [47]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [48]:
config={"configurable":{"session_id":"chat1"}}

In [49]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Yash and I am a software developer.")],
    config=config
)

In [50]:
response.content

"Hi Yash, it's nice to meet you!  \n\nI'm a large language model, here to help with any questions or tasks you might have. Since you're a software developer, is there anything specific I can assist you with today? Perhaps you need help brainstorming ideas, understanding a coding concept, or even just want to chat about the latest tech trends.  \n\nLet me know how I can be of service! 😊\n"

In [51]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Yash, as you told me at the beginning of our conversation. 😄  \n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 128, 'total_tokens': 160, 'completion_time': 0.058181818, 'prompt_time': 0.003519049, 'queue_time': 0.25026396, 'total_time': 0.061700867}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a268fd99-8a93-4100-9359-57d89ee3c50c-0', usage_metadata={'input_tokens': 128, 'output_tokens': 32, 'total_tokens': 160})

In [52]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know! 😊\n"

In [53]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today?\n"

In [54]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nI remember that from our last conversation. 😊  Is there anything else I can help you with?\n'

## Prompt Template

In [55]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt=ChatPromptTemplate.from_messages([
    ("system"," You are a helpful assistant that helps people find information."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [56]:
chain.invoke({"messages":[HumanMessage(content="Hii, My name is Yash")]})

AIMessage(content="Hello Yash! 👋 \n\nIt's nice to meet you.  \n\nWhat can I help you find information about today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 26, 'total_tokens': 58, 'completion_time': 0.058181818, 'prompt_time': 0.00140075, 'queue_time': 0.246009919, 'total_time': 0.059582568}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d7b37bd6-afc9-4f61-9608-3c1c8d6652cf-0', usage_metadata={'input_tokens': 26, 'output_tokens': 32, 'total_tokens': 58})

In [57]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [58]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="My name is Yash")],
    config=config
)
response.content

"Hello Yash!\n\nI'm glad to meet you.  \n\nWhat can I help you find today?  \n\nDo you have a specific question or topic in mind? I can search the web, provide summaries of topics, or even help you brainstorm ideas. Just let me know how I can be of assistance. 😊  \n\n"

In [59]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config
)
response.content

'Your name is Yash!  \n\nI remember that from our introduction. 😄  \n\nIs there anything else I can help you with, Yash?\n'

In [60]:
prompt=ChatPromptTemplate.from_messages([
    ("system"," You are a helpful assistant that helps people find information in {language}."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [61]:
response=chain.invoke(
    {"messages":[HumanMessage(content="Hii, My name is Yash")],
     "language":"Hindi"}
)

response.content

'नमस्ते Yash! 👋  \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। मैं आपकी मदद करने के लिए तैयार हूँ। \n\nआप क्या जानना चाहते हैं?  😊 \n\n'

In [62]:
chain.invoke(
    {"messages":[HumanMessage(content="What is Gen AI?")],"language":"English"}
)

AIMessage(content="Gen AI stands for **Generative Artificial Intelligence**. \n\nIt's a type of artificial intelligence that focuses on creating new content.  \n\nThink of it like this:\n\n* **Traditional AI** is good at recognizing patterns and making predictions based on existing data. For example, it can tell you what kind of weather to expect tomorrow based on historical weather patterns.\n\n* **Gen AI** takes it a step further. It learns from existing data but then uses that knowledge to generate brand new content. \n\n**Here are some examples of what Gen AI can do:**\n\n* **Write stories, poems, and articles:** \n\n* **Compose music and create art:**\n* **Translate languages:**\n* **Summarize text:**\n* **Answer your questions in a comprehensive and informative way, even if they are open ended, challenging, or strange:** (like this one!)\n\nGen AI is a rapidly developing field with huge potential to change the way we create and interact with information.\n", additional_kwargs={},

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [63]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [64]:
config={"configurable":{"session_id":"chat4"}}

response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Yash")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते यश! 👋 \n\nमैं आपकी मदद करने के लिए यहाँ हूँ।  आप मुझे कुछ भी पूछ सकते हैं और मैं अपनी पूरी कोशिश करूँगा कि आपको हिंदी में सही जानकारी मिले। 😊\n\nआप किस बारे में जानना चाहते हैं? \n'

In [65]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Whats my name?")],"language":"Hindi"},
    config=config
)
response.content

'आपका नाम यश है। 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages